In [6]:
#!/usr/bin/env python3
# code to replicate the performance of the NIPS'16 MLHC workshop paper "Cam CNNS predict anatomy?"

from __future__ import absolute_import
from __future__ import print_function

import os
import csv
import six

import numpy as np
import time
import json
import warnings

from collections import deque
from collections import OrderedDict
from collections import Iterable

# os.environ["THEANO_FLAGS"] = "device=gpu0,floatX=float32" 

os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ[
#     "THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32,force_device=true"#,lib.cnmem=0.7 ,nvcc.flags=-D_FORCE_INLINES"
import sys
import numpy as np
from keras import backend as K
K.set_image_data_format('channels_first')
#from keras.datasets import cifar10
#keras.callbacks.Callback()
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Convolution2D, MaxPooling2D 
from keras.layers import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, Callback
from keras.models import load_model
from keras.datasets import mnist
from scipy import io
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt


def load_data():
    data = np.load('/work/vsankar/Project-Luna/Train_nf_data.npz')
    X_train = data['X_train']
    Y_train = data['Y_train']
    
    data = np.load('/work/vsankar/Project-Luna/Test_nf_data.npz')
    X_test = data['X_test']
    Y_test = data['Y_test']    
    
    data = np.load('/work/vsankar/Project-Luna/Val_nf_data.npz')
    X_val = data['X_val']
    Y_val = data['Y_val']
    
    return X_train,Y_train,X_test,Y_test,X_val,Y_val

                    
def model_architecture(img_rows,img_cols,img_channels,nb_classes):
    #function defining the architecture of defined CNN
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True, input_shape=(img_channels,img_rows, img_cols)))
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2,2)))

    model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2,2)))

    model.add(Convolution2D(96, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(Convolution2D(96, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(Convolution2D(96, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2,2)))
    model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))
    Dropout((0.5))
    model.add(Convolution2D(512, 1, 1, activation='relu', border_mode='same',init='orthogonal', bias = True))
    Dropout((0.5))
    model.add(Convolution2D(2, 1, 1, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(GlobalAveragePooling2D(dim_ordering='default'))

    #model.add(Convolution2D(10,1,1, border_mode='same',init='orthogonal', bias = True))
    #model.add(Dense(nb_classes))

    model.add(Activation('softmax'))
    model.summary()
    return model


def normalize_date(X_train,Y_train,X_test,Y_test,X_val, Y_val,nb_classes):
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    print(X_val.shape[0], 'val samples')
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_val = X_val.astype('float32')
    
    Y_train = Y_train.astype('float32')
    Y_test = Y_test.astype('float32')
    Y_val = Y_val.astype('float32')

#     normalizing the data
    X_train /= 4095.0   
    X_test /= 4095.0
    X_val /= 4095.0
    
    #std
#     X_train = X_train/np.std(X_train) - np.mean(X_train)
#     X_test = X_test/np.std(X_test) - np.mean(X_test)
    
#     Tracer()()
    # convert class vectors to binary class matrices
#     Y_train = np_utils.to_categorical(Y_train, nb_classes)
#     Y_test = np_utils.to_categorical(Y_test, nb_classes)
#     Y_val = np_utils.to_categorical(Y_val, nb_classes)
    
    
    return X_train,Y_train,X_test,Y_test,X_val,Y_val

def run(batch_size,nb_classes,nb_epoch,data_augmentation,img_rows, img_cols,img_channels,model_name):

    
    
    print('Loading and formatiing of data complete...')
    
    #load the model defined in model_architecture function
    model = model_architecture(img_rows,img_cols,img_channels,nb_classes)

#     training the model using SGD + momentum
#     adm = Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#     model.compile(loss='categorical_crossentropy',
#                   optimizer=adm,
#                   metrics=['accuracy'])

  
    modelPath = '/work/vsankar/Project-Luna/Luna_weights/'+model_name+'.hdf5'
#     Tracer()()
    model.load_weights(modelPath)
#     Tracer()()
#     model.predict( X_test, batch_size=None, verbose=0, steps=None)

    return model

        # serialize model to JSON
        
model_name = 'luna_weights_t_ae5_nt_b128_d25_BN0_best'     
batch_size = 50
nb_classes = 2
nb_epoch = 30
data_augmentation = True
# input image dimensions
img_rows, img_cols = 96,96
# the imgCLEF images are grey
img_channels = 1
model = run(batch_size,nb_classes,nb_epoch,data_augmentation,img_rows,img_cols,img_channels,model_name)
# model_architecture(img_rows,img_cols,img_channels,nb_classes)

Loading and formatiing of data complete...


/home/vsankar/pyenvs/LungCancer/lib/python2.7/site-packages/ipykernel/__main__.py:71: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="orthogonal", activation="relu", input_shape=(1, 96, 96..., padding="same", use_bias=True)`
/home/vsankar/pyenvs/LungCancer/lib/python2.7/site-packages/ipykernel/__main__.py:72: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="orthogonal", activation="relu", padding="same", use_bias=True)`
/home/vsankar/pyenvs/LungCancer/lib/python2.7/site-packages/ipykernel/__main__.py:73: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="orthogonal", activation="relu", padding="same", use_bias=True)`
/home/vsankar/pyenvs/LungCancer/lib/python2.7/site-packages/ipykernel/__main__.py:77: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), kernel_initializer="orthogonal", activation="relu", padding="

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 32, 96, 96)        320       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 32, 96, 96)        9248      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 32, 96, 96)        9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 48, 48)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 64, 48, 48)        18496     
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 64, 48, 48)        36928     
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 64, 48, 48)        36928     
__________

In [7]:
X_test,y_test,X_train,y_train,x_val,y_val = load_data()
nb_classes=2
X_train,Y_train,X_test,Y_test,X_val,Y_val = normalize_date(X_test,y_test,X_train,y_train,x_val,y_val,nb_classes)
# np.save('/work/vsankar/Project-Luna/Test_data.npy', X_test,Y_test)


X_testing = X_val
Y_testing = Y_val


y = model.predict( X_testing, batch_size=1, verbose=0)

X_train shape: (35191, 1, 96, 96)
35191 train samples
2997 test samples
2997 val samples


In [13]:
Y_test[:,].astype('float32')


array([ 1.,  1.,  1., ...,  0.,  0.,  0.], dtype=float32)

In [14]:
 y[:,1]

array([ 0.96031159,  0.95115805,  0.96130818, ...,  0.12470222,
        0.6728462 ,  0.58903307], dtype=float32)

In [8]:
from sklearn import metrics as m
from IPython.core.debugger import Tracer;
Threshold = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
i=0
sen_tpr=np.zeros(len(Threshold))
spc_tnr=np.zeros(len(Threshold))
acc=np.zeros(len(Threshold))
tn=np.zeros(len(Threshold))
fp=np.zeros(len(Threshold))
fn=np.zeros(len(Threshold))
tp= np.zeros(len(Threshold))
for t in Threshold:
#      Tracer()() 
    y_pred = y.copy()
    y_true = Y_test
    y_pred[y_pred>=t]=1
#     Tracer()()
    y_pred[y_pred<t] = 0
    
    cm = m.confusion_matrix(y_true[:,1], y_pred[:,1])
#     Tracer()()
    
    tn[i], fp[i], fn[i], tp[i] = cm.ravel().astype('float32')
    
    
    sen_tpr[i]=tp[i]/(tp[i]+fn[i])
    spc_tnr[i] = tn[i]/(tn[i]+fp[i])
    acc[i] = (tp[i] +tn[i])/(tn[i]+tp[i]+fp[i]+fn[i])
#     from IPython.core.debugger import Tracer; Tracer()() 
    i=i+1
    


In [9]:
sen_tpr


array([ 0.94477998,  0.92407248,  0.90638481,  0.8852459 ,  0.85893011,
        0.82398619,  0.76747196,  0.63977567,  0.42622951])

In [24]:
tn,tp,fp,fn


(array([ 209.,  230.,  259.,  275.,  285.,  300.,  330.,  370.,  449.]),
 array([ 1950.,  1943.,  1939.,  1929.,  1891.,  1824.,  1705.,  1555.,
         1308.]),
 array([ 332.,  311.,  282.,  266.,  256.,  241.,  211.,  171.,   92.]),
 array([  19.,   26.,   30.,   40.,   78.,  145.,  264.,  414.,  661.]))

In [10]:
spc_tnr


array([ 0.05007364,  0.06332842,  0.08541973,  0.10751105,  0.13402062,
        0.16494845,  0.22827688,  0.3460972 ,  0.57437408])

In [13]:
acc


array([ 0.74207541,  0.7290624 ,  0.72038705,  0.70904238,  0.69469469,
        0.67467467,  0.64531198,  0.57323991,  0.45979313])

In [19]:
ypint = np.packbits(y.astype('int'),axis=-1)
ytrueint = np.packbits(Y_test.astype('int'),axis=-1)

In [ ]:
# y.astype('float32'),Y_test.astype('float32')
ypint,ytrueint


In [ ]:
from sklearn import metrics as m
print( "accuracy is = " )
print (m.accuracy_score(ypint, ytrueint))
print( "confusion matrix is ")
print (m.confusion_matrix( ytrueint,ypint))
print ("roc is ")
fpr, tpr, thresholds = m.roc_curve(ytrueint, ypint, pos_label=2)
print("fpr = ")
print(fpr),
print(" tpr = " )
print(tpr)
print(" thresholds = ")
print(thresholds)
roc_auc = m.auc(fpr, tpr)

In [ ]:
y_test.ravel()

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
Y_test[1:200,:]

In [ ]:
np.save('/work/vsankar/Project-Luna/Test_data.npy', X_test,Y_test)


In [ ]:
data = np.load('/work/vsankar/Project-Luna/Test_data.npy')

In [ ]:
X_test,y_test,X_train,y_train = load_data()
# X_train,Y_train,X_test,Y_test = normalize_date(X_train,y_train,X_test,y_test,nb_classes)

In [ ]:
y_train>1


In [ ]:
 np.count_nonzero(y_test) - 2510
    

In [ ]:
y_test[1:10]